# Data Cleaning and Preparation

During the course of doing data analysis and modeling, a significant amount of time
is spent on data preparation: loading, cleaning, transforming, and rearranging. Such
tasks are often reported to take up 80% or more of an analyst’s time. Sometimes the
way that data is stored in files or databases is not in the right format for a particular
task. Many researchers choose to do ad hoc processing of data from one form to
another using a general-purpose programming language, like Python, Perl, R, or Java,
or Unix text-processing tools like sed or awk. Fortunately, pandas, along with the
built-in Python language features, provides you with a high-level, flexible, and fast set
of tools to enable you to manipulate data into the right form.

In this class, we will discuss tools for missing data, duplicate data, string manipulation,
and some other analytical data transformations.

In [75]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## Handling Missing Data

Missing data occurs commonly in many data analysis applications. One of the goals
of pandas is to make working with missing data as painless as possible. For example,
all of the descriptive statistics on pandas objects exclude missing data by default.

The way that missing data is represented in pandas objects is somewhat imperfect,
but it is functional for a lot of users. For numeric data, pandas uses the floating-point
value NaN (Not a Number) to represent missing data. We call this a sentinel value that
can be easily detected:

In [76]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado', np.nan])
string_data
string_data.isnull()

0    False
1    False
2     True
3    False
4     True
dtype: bool

In [4]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### Filtering Out Missing Data

In pandas, we’ve adopted a convention used in the R programming language by referring
to missing data as NA, which stands for not available. In statistics applications,
NA data may either be data that does not exist or that exists but was not observed
(through problems with data collection, for example). When cleaning up data for
analysis, it is often important to do analysis on the missing data itself to identify data
collection problems or potential biases in the data caused by missing data.

The built-in Python None value is also treated as NA in object arrays:

There are a few ways to filter out missing data. While you always have the option to
do it by hand using pandas.isnull and boolean indexing, the dropna can be helpful.
On a Series, it returns the Series with only the non-null data and index values:

In [5]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

This is equivalent to:

In [6]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

With DataFrame objects, things are a bit more complex. You may want to drop rows
or columns that are all NA or only those containing any NAs. dropna by default drops
any row containing a missing value:

In [7]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [8]:
cleaned

,0,1,2
0,1.0,6.5,3.0


Passing how='all' will only drop rows that are all NA:

In [9]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


To drop columns in the same way, pass axis=1:

In [10]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [11]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


Suppose we want to keep only rows containing a certain number of observations. We can
indicate this with the thresh argument:

In [12]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [13]:
df.dropna()

,0,1,2
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [14]:
df.dropna(thresh=2)

,0,1,2
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


### Filling In Missing Data

Rather than filtering out missing data (and potentially discarding other data along
with it), you may want to fill in the “holes” in any number of ways. For most purposes,
the fillna method is the workhorse function to use. Calling fillna with a
constant replaces missing values with that value:

In [18]:
df

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [15]:
df.fillna(0)

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [19]:
df.fillna(method='bfill')

,0,1,2
0,-0.204708,0.228913,0.769023
1,-0.555730,0.228913,0.769023
2,0.092908,0.228913,0.769023
3,1.246435,0.228913,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


Calling fillna with a dict, you can use a different fill value for each column:

In [16]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.204708,0.500000,0.000000
1,-0.555730,0.500000,0.000000
2,0.092908,0.500000,0.769023
3,1.246435,0.500000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


fillna returns a new object, but you can modify the existing object in-place:

In [17]:
_ = df.fillna(0, inplace=True)
df

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


The same interpolation methods available for reindexing can be used with fillna:

In [18]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,NaN,1.343810
3,-0.713544,NaN,-2.370232
4,-1.860761,NaN,NaN
5,-1.265934,NaN,NaN


In [16]:
df.fillna(method='ffill')

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [17]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


With fillna you can do lots of other things with a little creativity. For example, we
could pass the mean or median value of a Series:

In [21]:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## Data Transformation

Filtering, cleaning,
and other transformations are another class of important operations.

### Removing Duplicates

Duplicate rows may be found in a DataFrame for any number of reasons. Here is an
example:

In [3]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


The DataFrame method duplicated returns a boolean Series indicating whether each
row is a duplicate (has been observed in a previous row) or not:

In [4]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

Relatedly, drop_duplicates returns a DataFrame where the duplicated array is
False:

In [5]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Both of these methods by default consider all of the columns; alternatively, you can
specify any subset of them to detect duplicates. Suppose we had an additional column
of values and wanted to filter duplicates only based on the 'k1' column:

In [6]:
data['v1'] = range(7)
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


duplicated and drop_duplicates by default keep the first observed value combination.
Passing keep='last' will return the last one:

In [7]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transforming Data Using a Function or Mapping

For many datasets, you may wish to perform some transformation based on the values
in an array, Series, or column in a DataFrame. Consider the following hypothetical
data collected about various kinds of meat:

In [13]:
data = pd.DataFrame({'food': ['Pastrami', 'corned beef',
                              'pastrami', 'nova lox'],
                     'ounces': [4, 3, 12, 6]})
data

,food,ounces
0,Pastrami,4
1,corned beef,3
2,pastrami,12
3,nova lox,6


Suppose you wanted to add a column indicating the type of animal that each food
came from. Let’s write down a mapping of each distinct meat type to the kind of
animal:

In [14]:
meat_to_animal = {
  'pastrami': 'cow',
  'corned beef': 'cow',
  'nova lox': 'salmon'
}

The map method on a Series accepts a function or dict-like object containing a mapping,
but here we have a small problem in that some of the meats are capitalized and
others are not. Thus, we need to convert each value to lowercase using the str.lower
Series method:

In [15]:
lowercased = data['food'].str.lower()
lowercased
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,Pastrami,4,cow
1,corned beef,3,cow
2,pastrami,12,cow
3,nova lox,6,salmon


We could also have passed a function that does all the work:

In [16]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       cow
1       cow
2       cow
3    salmon
Name: food, dtype: object

Using map is a convenient way to perform element-wise transformations and other
data cleaning–related operations.

### Replacing Values

Filling in missing data with the fillna method is a special case of more general value
replacement. As you’ve already seen, map can be used to modify a subset of values in
an object but replace provides a simpler and more flexible way to do so. Let’s consider
this Series:

In [17]:
data = pd.Series([1., -999., 2., 
                  -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

The -999 values might be sentinel values for missing data. To replace these with NA
values that pandas understands, we can use replace, producing a new Series (unless
you pass inplace=True):

In [18]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

If you want to replace multiple values at once, you instead pass a list and then the
substitute value:

In [19]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

To use a different replacement for each value, pass a list of substitutes:

In [20]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

The argument passed can also be a dict:

In [21]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes

Like values in a Series, axis labels can be similarly transformed by a function or mapping
of some form to produce new, differently labeled objects. You can also modify
the axes in-place without creating a new data structure. Here’s a simple example:

In [27]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Porto', 'Lisboa', 'Alentenjo'],
                    columns=['one', 'two', 'three', 'four'])

Like a Series, the axis indexes have a map method:

In [22]:
transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['PORT', 'LISB', 'ALEN'], dtype='object')

You can assign to index, modifying the DataFrame in-place:

In [23]:
data.index = data.index.map(transform)
data

,one,two,three,four
PORT,0,1,2,3
LISB,4,5,6,7
ALEN,8,9,10,11


If you want to create a transformed version of a dataset without modifying the original,
a useful method is rename:

In [24]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Port,0,1,2,3
Lisb,4,5,6,7
Alen,8,9,10,11


Notably, rename can be used in conjunction with a dict-like object providing new values
for a subset of the axis labels

In [30]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Porto', 'Lisboa', 'Alentenjo'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Porto,0,1,2,3
Lisboa,4,5,6,7
Alentenjo,8,9,10,11


In [29]:
data.rename(index={'Lisboa': 'Cascais'},
            columns={'three': 'threeandhalf'})

,one,two,threeandhalf,four
Porto,0,1,2,3
Cascais,4,5,6,7
Alentenjo,8,9,10,11


rename saves you from the chore of copying the DataFrame manually and assigning
to its index and columns attributes. Should you wish to modify a dataset in-place,
pass inplace=True:

In [26]:
data.rename(index={'Porto': 'Braga'}, inplace=True)
data

,one,two,three,four
PORT,0,1,2,3
LISB,4,5,6,7
ALEN,8,9,10,11


### Discretization and Binning
Continuous data is often discretized or otherwise separated into “bins” for analysis.
Suppose you have data about a group of people in a study, and you want to group
them into discrete age buckets:

In [33]:
ages = [20, 22, 25, 27, 21, 23, 
        37, 31, 61, 45, 41, 32]

Let’s divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older. To
do so, you have to use cut, a function in pandas:

In [34]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

The object pandas returns is a special Categorical object. The output you see
describes the bins computed by pandas.cut. You can treat it like an array of strings
indicating the bin name; internally it contains a categories array specifying the distinct
category names along with a labeling for the ages data in the codes attribute:

In [35]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [36]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]]
              closed='right',
              dtype='interval[int64]')

In [37]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

Note that pd.value_counts(cats) are the bin counts for the result of pandas.cut.
Consistent with mathematical notation for intervals, a parenthesis means that the side
is open, while the square bracket means it is closed (inclusive). You can change which
side is closed by passing right=False:

In [32]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

You can also pass your own bin names by passing a list or array to the labels option:

In [33]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

If you pass an integer number of bins to cut instead of explicit bin edges, it will compute
equal-length bins based on the minimum and maximum values in the data.
Consider the case of some uniformly distributed data chopped into fourths:

In [38]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2)

[(0.73, 0.96], (0.25, 0.49], (0.0074, 0.25], (0.0074, 0.25], (0.49, 0.73], ..., (0.49, 0.73], (0.73, 0.96], (0.73, 0.96], (0.73, 0.96], (0.49, 0.73]]
Length: 20
Categories (4, interval[float64]): [(0.0074, 0.25] < (0.25, 0.49] < (0.49, 0.73] < (0.73, 0.96]]

In [39]:
pd.cut(ages, 4, precision=2)

[(19.96, 30.25], (19.96, 30.25], (19.96, 30.25], (19.96, 30.25], (19.96, 30.25], ..., (30.25, 40.5], (50.75, 61.0], (40.5, 50.75], (40.5, 50.75], (30.25, 40.5]]
Length: 12
Categories (4, interval[float64]): [(19.96, 30.25] < (30.25, 40.5] < (40.5, 50.75] < (50.75, 61.0]]

The precision=2 option limits the decimal precision to two digits.
A closely related function, qcut, bins the data based on sample quantiles. Depending
on the distribution of the data, using cut will not usually result in each bin having the
same number of data points. Since qcut uses sample quantiles instead, by definition
you will obtain roughly equal-size bins:

In [41]:
data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data, 4)  # Cut into quartiles
cats

[(-0.0189, 0.681], (-3.746, -0.623], (-0.0189, 0.681], (0.681, 3.26], (-0.623, -0.0189], ..., (0.681, 3.26], (-3.746, -0.623], (-0.0189, 0.681], (0.681, 3.26], (-3.746, -0.623]]
Length: 1000
Categories (4, interval[float64]): [(-3.746, -0.623] < (-0.623, -0.0189] < (-0.0189, 0.681] < (0.681, 3.26]]

In [42]:
pd.value_counts(cats)

(0.681, 3.26]        250
(-0.0189, 0.681]     250
(-0.623, -0.0189]    250
(-3.746, -0.623]     250
dtype: int64

Similar to cut you can pass your own quantiles (numbers between 0 and 1, inclusive):

In [43]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(-0.0189, 1.338], (-3.746, -1.296], (-0.0189, 1.338], (1.338, 3.26], (-1.296, -0.0189], ..., (-0.0189, 1.338], (-1.296, -0.0189], (-0.0189, 1.338], (1.338, 3.26], (-1.296, -0.0189]]
Length: 1000
Categories (4, interval[float64]): [(-3.746, -1.296] < (-1.296, -0.0189] < (-0.0189, 1.338] < (1.338, 3.26]]

In [50]:
pd.qcut(ages, [0, 0.2, 0.5, 1.])

[(19.999, 22.2], (19.999, 22.2], (22.2, 29.0], (22.2, 29.0], (19.999, 22.2], ..., (29.0, 61.0], (29.0, 61.0], (29.0, 61.0], (29.0, 61.0], (29.0, 61.0]]
Length: 12
Categories (3, interval[float64]): [(19.999, 22.2] < (22.2, 29.0] < (29.0, 61.0]]

### Detecting and Filtering Outliers

Filtering or transforming outliers is largely a matter of applying array operations.
Consider a DataFrame with some normally distributed data:

In [37]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.021425,-0.005165,-0.055670,0.044107
std,1.008404,0.995532,0.994484,0.996884
min,-3.184377,-3.745356,-3.428254,-3.645860
25%,-0.628122,-0.699383,-0.747478,-0.599807
50%,-0.016127,-0.029924,-0.091364,0.043663
75%,0.690847,0.694459,0.620197,0.740562
max,3.525865,2.735527,3.366626,2.653656


Suppose you wanted to find values in one of the columns exceeding 3 in absolute
value:

In [38]:
col = data[2]
col[np.abs(col) > 3]

269   -3.428254
646    3.366626
Name: 2, dtype: float64

To select all rows having a value exceeding 3 or –3, you can use the any method on a
boolean DataFrame:

In [39]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
52,-0.025907,-3.399312,-0.974657,-0.685312
71,3.260383,0.963301,1.201206,-1.852001
147,-0.196713,-3.745356,-1.520113,-0.346839
246,-3.056990,1.918403,-0.578828,1.847446
269,0.326045,0.425384,-3.428254,-0.296336
333,-3.184377,1.369891,-1.074833,-0.089937
554,0.208011,-0.150923,-0.362528,-3.548824
646,0.193299,1.397822,3.366626,-2.372214
793,3.525865,0.283070,0.544635,0.462204
813,-0.450721,-0.080332,0.599947,-3.645860


Values can be set based on these criteria. Here is code to cap values outside the interval
–3 to 3:

In [40]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.020880,-0.004021,-0.055608,0.045301
std,1.005160,0.991718,0.991924,0.992863
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.628122,-0.699383,-0.747478,-0.599807
50%,-0.016127,-0.029924,-0.091364,0.043663
75%,0.690847,0.694459,0.620197,0.740562
max,3.000000,2.735527,3.000000,2.653656


The statement np.sign(data) produces 1 and –1 values based on whether the values
in data are positive or negative:

In [41]:
np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,1.0,1.0
1,-1.0,-1.0,-1.0,-1.0
2,1.0,1.0,1.0,-1.0
3,1.0,1.0,-1.0,-1.0
4,1.0,-1.0,-1.0,-1.0


### Permutation and Random Sampling

Permuting (randomly reordering) a Series or the rows in a DataFrame is easy to do
using the numpy.random.permutation function. Calling permutation with the length
of the axis you want to permute produces an array of integers indicating the new
ordering:

In [42]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
sampler = np.random.permutation(5)
sampler

array([1, 0, 4, 2, 3])

That array can then be used in iloc-based indexing or the equivalent take function:

In [43]:
df
df.take(sampler)

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
4,16,17,18,19
2,8,9,10,11
3,12,13,14,15


To select a random subset without replacement, you can use the sample method on
Series and DataFrame:

In [44]:
df.sample(n=3)

,0,1,2,3
0,0,1,2,3
2,8,9,10,11
1,4,5,6,7


To generate a sample with replacement (to allow repeat choices), pass replace=True
to sample:

In [45]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

0    5
3    6
1    7
4    4
0    5
3    6
0    5
3    6
2   -1
3    6
dtype: int64

### Computing Indicator/Dummy Variables

Another type of transformation for statistical modeling or machine learning applications
is converting a categorical variable into a “dummy” or “indicator” matrix. If a
column in a DataFrame has k distinct values, you would derive a matrix or Data‐
Frame with k columns containing all 1s and 0s. pandas has a get_dummies function
for doing this, though devising one yourself is not difficult. Let’s return to an earlier
example DataFrame:

In [53]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 
                           'c', 'a', 'b'],
                   'data1': range(6)})
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In some cases, you may want to add a prefix to the columns in the indicator DataFrame, which can then be merged with the other data. get_dummies has a prefix argument
for doing this:

In [56]:
dummies = pd.get_dummies(df['key'], prefix='key')
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [57]:
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


## String Manipulation

Python has long been a popular raw data manipulation language in part due to its
ease of use for string and text processing. Most text operations are made simple with
the string object’s built-in methods. For more complex pattern matching and text
manipulations, regular expressions may be needed. pandas adds to the mix by enabling
you to apply string and regular expressions concisely on whole arrays of data,
additionally handling the annoyance of missing data.

### String Object Methods

In many string munging and scripting applications, built-in string methods are sufficient.
As an example, a comma-separated string can be broken into pieces with
split:

In [56]:
val = 'a,b,  guido'
val.split(',')

['a', 'b', '  guido']

split is often combined with strip to trim whitespace (including line breaks):

In [58]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

These substrings could be concatenated together with a two-colon delimiter using
addition:

In [59]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

But this isn’t a practical generic method. A faster and more Pythonic way is to pass a
list or tuple to the join method on the string '::':

In [60]:
'::'.join(pieces)

'a::b::guido'

Other methods are concerned with locating substrings. Using Python’s in keyword is
the best way to detect a substring, though index and find can also be used:

In [61]:
'guido' in val

-1

In [62]:
val.index(',')

1

In [63]:
val.find(':')

-1

Note the difference between find and index is that index raises an exception if the
string isn’t found (versus returning –1):

In [64]:
val.index(':')

ValueError: substring not found

Relatedly, count returns the number of occurrences of a particular substring:

In [65]:
val.count(',')

2

replace will substitute occurrences of one pattern for another. It is commonly used
to delete patterns, too, by passing an empty string:

In [66]:
val.replace(',', '::')
val.replace(',', '')

'ab  guido'

### Vectorized String Functions in pandas

Cleaning up a messy dataset for analysis often requires a lot of string munging and
regularization. To complicate matters, a column containing strings will sometimes
have missing data:

In [69]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [68]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

You can apply string and regular expression methods can be applied (passing a
lambda or other function) to each value using data.map, but it will fail on the NA
(null) values. To cope with this, Series has array-oriented methods for string operations
that skip NA values. These are accessed through Series’s str attribute; for example,
we could check whether each email address has 'gmail' in it with str.contains:

In [69]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

You can similarly slice strings using this syntax:

In [72]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

In [70]:
data.str.upper()

Dave     DAVE@GOOGLE.COM
Steve    STEVE@GMAIL.COM
Rob        ROB@GMAIL.COM
Wes                  NaN
dtype: object

In [71]:
data.str.isalpha()

Dave     False
Steve    False
Rob      False
Wes        NaN
dtype: object

In [72]:
data.str.replace('gmail', 'yahoo')

Dave     dave@google.com
Steve    steve@yahoo.com
Rob        rob@yahoo.com
Wes                  NaN
dtype: object

In [74]:
data.str.len()

Dave     15.0
Steve    15.0
Rob      13.0
Wes       NaN
dtype: float64